#### ETL Project
#### Contributors:
Matheus Gratz / Roberto Gonzalez / Sergio Guarneros

In [1]:
# Import dependencies
import pandas as pd
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [2]:
# Import credentials
from my_credentials import PGHOST, PGDATABASE, PGUSER, PGPASSWORD

## Read CSV for 'circuits', load it to a dataframe and do some transformations

In [3]:
path_circuits = 'Resources/circuits.csv'
circuits_df = pd.read_csv(path_circuits, encoding='latin_1')
circuits_df.head()

,circuitId,circuitRef,name,location,country,lat,lng,alt,url
0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.96800,10.0,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
1,2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.73800,NaN,http://en.wikipedia.org/wiki/Sepang_Internatio...
2,3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.03250,50.51060,NaN,http://en.wikipedia.org/wiki/Bahrain_Internati...
3,4,catalunya,Circuit de Barcelona-Catalunya,MontmelÌ_,Spain,41.57000,2.26111,NaN,http://en.wikipedia.org/wiki/Circuit_de_Barcel...
4,5,istanbul,Istanbul Park,Istanbul,Turkey,40.95170,29.40500,NaN,http://en.wikipedia.org/wiki/Istanbul_Park


In [4]:
#Drop 'alt' columns because we don't have all data for all circuits and the 'url'
circuits_df = circuits_df[['circuitId', 'circuitRef', 'name', 'location', 'country', 'lat', 'lng']]
circuits_df.head()

,circuitId,circuitRef,name,location,country,lat,lng
0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.96800
1,2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.73800
2,3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.03250,50.51060
3,4,catalunya,Circuit de Barcelona-Catalunya,MontmelÌ_,Spain,41.57000,2.26111
4,5,istanbul,Istanbul Park,Istanbul,Turkey,40.95170,29.40500


## Read CSV for 'drivers', load it to a dataframe and do some transformations

In [5]:
path_drivers = 'Resources/drivers.csv'
drivers_df = pd.read_csv(path_drivers, encoding='latin_1')
drivers_df.head()

,driverId,driverRef,number,code,forename,surname,dob,nationality,url
0,1,hamilton,44.0,HAM,Lewis,Hamilton,07/01/1985,British,http://en.wikipedia.org/wiki/Lewis_Hamilton
1,2,heidfeld,NaN,HEI,Nick,Heidfeld,10/05/1977,German,http://en.wikipedia.org/wiki/Nick_Heidfeld
2,3,rosberg,6.0,ROS,Nico,Rosberg,27/06/1985,German,http://en.wikipedia.org/wiki/Nico_Rosberg
3,4,alonso,14.0,ALO,Fernando,Alonso,29/07/1981,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso
4,5,kovalainen,NaN,KOV,Heikki,Kovalainen,19/10/1981,Finnish,http://en.wikipedia.org/wiki/Heikki_Kovalainen


In [6]:
#Drop 'numbers' columns because we don't have all data for all circuits and the 'url'
drivers_df = drivers_df[['driverId', 'driverRef', 'code', 'forename', 'surname', 'dob', 'nationality']]
drivers_df.head()

,driverId,driverRef,code,forename,surname,dob,nationality
0,1,hamilton,HAM,Lewis,Hamilton,07/01/1985,British
1,2,heidfeld,HEI,Nick,Heidfeld,10/05/1977,German
2,3,rosberg,ROS,Nico,Rosberg,27/06/1985,German
3,4,alonso,ALO,Fernando,Alonso,29/07/1981,Spanish
4,5,kovalainen,KOV,Heikki,Kovalainen,19/10/1981,Finnish


## Read CSV for 'races', load it to a dataframe and do some transformations

In [7]:
path_races = 'Resources/races.csv'
races_df = pd.read_csv(path_races, encoding='latin_1')
races_df.head()

,raceId,year,round,circuitId,name,date,time,url
0,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...
1,2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...
2,3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Gran...
3,4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Gran...
4,5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Gran...


In [8]:
races_df.columns

Index(['raceId', 'year', 'round', 'circuitId', 'name', 'date', 'time', 'url'], dtype='object')

In [9]:
#Drop 'time' and 'url' columns because we don't need this info 
races_df = races_df[['raceId', 'year', 'round', 'circuitId', 'name', 'date']]
races_df.head()

,raceId,year,round,circuitId,name,date
0,1,2009,1,1,Australian Grand Prix,2009-03-29
1,2,2009,2,2,Malaysian Grand Prix,2009-04-05
2,3,2009,3,17,Chinese Grand Prix,2009-04-19
3,4,2009,4,3,Bahrain Grand Prix,2009-04-26
4,5,2009,5,4,Spanish Grand Prix,2009-05-10


## Read CSV for 'results', load it to a dataframe and do some transformations

In [10]:
path_results = 'Resources/results.csv'
results_df = pd.read_csv(path_results, encoding='latin_1')
results_df.head()

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22.0,1,1.0,1,1,10.0,58,34:50.6,5690616.0,39.0,2.0,01:27.5,218.3,1
1,2,18,2,2,3.0,5,2.0,2,2,8.0,58,5.478,5696094.0,41.0,3.0,01:27.7,217.586,1
2,3,18,3,3,7.0,7,3.0,3,3,6.0,58,8.163,5698779.0,41.0,5.0,01:28.1,216.719,1
3,4,18,4,4,5.0,11,4.0,4,4,5.0,58,17.181,5707797.0,58.0,7.0,01:28.6,215.464,1
4,5,18,5,1,23.0,3,5.0,5,5,4.0,58,18.014,5708630.0,43.0,1.0,01:27.4,218.385,1


In [11]:
#Drop 'constructorId', 'number', 'positionText', 'positionOrder', 'rank', and 'statusId' columns because we don't need this info 
results_df = results_df[['resultId', 'raceId', 'driverId',  'grid', 'position',  'points', 'laps', 'time','milliseconds', 'fastestLap', 'fastestLapTime', 'fastestLapSpeed']]
results_df.head()

,resultId,raceId,driverId,grid,position,points,laps,time,milliseconds,fastestLap,fastestLapTime,fastestLapSpeed
0,1,18,1,1,1.0,10.0,58,34:50.6,5690616.0,39.0,01:27.5,218.3
1,2,18,2,5,2.0,8.0,58,5.478,5696094.0,41.0,01:27.7,217.586
2,3,18,3,7,3.0,6.0,58,8.163,5698779.0,41.0,01:28.1,216.719
3,4,18,4,11,4.0,5.0,58,17.181,5707797.0,58.0,01:28.6,215.464
4,5,18,5,3,5.0,4.0,58,18.014,5708630.0,43.0,01:27.4,218.385


## Create Database Connection

In [12]:
conn = f'postgresql://{PGUSER}:{PGPASSWORD}@{PGHOST}:5432/{PGDATABASE}'
engine = create_engine(conn)
connection = engine.connect()

In [13]:
# Confirm tables
inspector = inspect(engine)
databases = inspector.get_table_names()
print(databases)

['drivers', 'results', 'circuits', 'races']


In [14]:
# Declare a Base using `automap_base()`
Base = automap_base()

In [15]:
# Use the Base class to reflect the database tables
Base.prepare(engine, reflect=True)

In [16]:
# Print all of the classes mapped to the Base
Base.classes.keys()

['drivers', 'results', 'races', 'circuits']

## Load dataframes into Database

In [17]:
circuits_df.to_sql(name='circuits', con=engine, if_exists='append', index=False)

In [18]:
drivers_df.to_sql(name='drivers', con=engine, if_exists='append', index=False)

In [19]:
races_df.to_sql(name='races', con=engine, if_exists='append', index=False)

In [20]:
results_df.to_sql(name='results', con=engine, if_exists='append', index=False)

## Check if everything was loaded correctly

In [21]:
pd.read_sql('SELECT * FROM circuits LIMIT(10)', engine)

,circuitId,circuitRef,name,location,country,lat,lng
0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.96800
1,2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.73800
2,3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.03250,50.51060
3,4,catalunya,Circuit de Barcelona-Catalunya,MontmelÌ_,Spain,41.57000,2.26111
4,5,istanbul,Istanbul Park,Istanbul,Turkey,40.95170,29.40500
5,6,monaco,Circuit de Monaco,Monte-Carlo,Monaco,43.73470,7.42056
6,7,villeneuve,Circuit Gilles Villeneuve,Montreal,Canada,45.50000,-73.52280
7,8,magny_cours,Circuit de Nevers Magny-Cours,Magny Cours,France,46.86420,3.16361
8,9,silverstone,Silverstone Circuit,Silverstone,UK,52.07860,-1.01694
9,10,hockenheimring,Hockenheimring,Hockenheim,Germany,49.32780,8.56583


In [22]:
pd.read_sql('SELECT * FROM drivers LIMIT(10)', engine)

,driverId,driverRef,code,forename,surname,dob,nationality
0,1,hamilton,HAM,Lewis,Hamilton,1985-01-07,British
1,2,heidfeld,HEI,Nick,Heidfeld,1977-05-10,German
2,3,rosberg,ROS,Nico,Rosberg,1985-06-27,German
3,4,alonso,ALO,Fernando,Alonso,1981-07-29,Spanish
4,5,kovalainen,KOV,Heikki,Kovalainen,1981-10-19,Finnish
5,6,nakajima,NAK,Kazuki,Nakajima,1985-01-11,Japanese
6,7,bourdais,BOU,SÌ©bastien,Bourdais,1979-02-28,French
7,8,raikkonen,RAI,Kimi,RÌ_ikkÌ¦nen,1979-10-17,Finnish
8,9,kubica,KUB,Robert,Kubica,1984-12-07,Polish
9,10,glock,GLO,Timo,Glock,1982-03-18,German


In [23]:
pd.read_sql('SELECT * FROM races LIMIT(10)', engine)

,raceId,year,round,circuitId,name,date
0,1,2009,1,1,Australian Grand Prix,2009-03-29
1,2,2009,2,2,Malaysian Grand Prix,2009-04-05
2,3,2009,3,17,Chinese Grand Prix,2009-04-19
3,4,2009,4,3,Bahrain Grand Prix,2009-04-26
4,5,2009,5,4,Spanish Grand Prix,2009-05-10
5,6,2009,6,6,Monaco Grand Prix,2009-05-24
6,7,2009,7,5,Turkish Grand Prix,2009-06-07
7,8,2009,8,9,British Grand Prix,2009-06-21
8,9,2009,9,20,German Grand Prix,2009-07-12
9,10,2009,10,11,Hungarian Grand Prix,2009-07-26


In [24]:
pd.read_sql('SELECT * FROM results LIMIT(10)', engine)

,resultId,raceId,driverId,grid,position,points,laps,time,milliseconds,fastestLap,fastestLapTime,fastestLapSpeed
0,1,18,1,1,1.0,10,58,34:50.6,5690616.0,39.0,01:27.5,218.3
1,2,18,2,5,2.0,8,58,5.478,5696094.0,41.0,01:27.7,217.586
2,3,18,3,7,3.0,6,58,8.163,5698779.0,41.0,01:28.1,216.719
3,4,18,4,11,4.0,5,58,17.181,5707797.0,58.0,01:28.6,215.464
4,5,18,5,3,5.0,4,58,18.014,5708630.0,43.0,01:27.4,218.385
5,6,18,6,13,6.0,3,57,None,None,50.0,01:29.6,212.974
6,7,18,7,17,7.0,2,55,None,None,22.0,01:29.5,213.224
7,8,18,8,15,8.0,1,53,None,None,20.0,01:27.9,217.18
8,9,18,9,2,NaN,0,47,None,None,15.0,01:28.8,215.1
9,10,18,10,18,NaN,0,43,None,None,23.0,01:29.6,213.166


## Answering some questions...

In [25]:
# Assign tables to variables - just the one that will be used
drivers_db = Base.classes.drivers
circuits_db = Base.classes.circuits
races_db = Base.classes.races
results_db = Base.classes.results

In [26]:
# Create a session
session = Session(engine)

In [27]:
# What's the distribution of drivers based on their nationality?
nationality_result = session.query(drivers_db.nationality, func.count(drivers_db.driverId)).\
    group_by(drivers_db.nationality).\
    order_by(func.count(drivers_db.driverId).desc()).all()
nationality_df = pd.DataFrame(nationality_result, columns=['nationality', 'count'])
nationality_df.set_index('nationality', inplace=True)
nationality_df.head(10)

,count
nationality,
British,162
American,157
Italian,99
French,73
German,49
Brazilian,31
Argentine,24
South African,23
Belgian,23


In [37]:
# Based on the nationality: What's the distribution of winners, 2nd and 3rd places?
winners_results = session.query(drivers_db.nationality, func.count(drivers_db.driverId)).\
    filter(drivers_db.driverId == results_db.driverId).\
    filter(results_db.position == 1).\
    group_by(drivers_db.nationality).\
    order_by(func.count(drivers_db.driverId).desc()).all()
winners_results_df = pd.DataFrame(winners_results, columns=['nationality', 'count'])
winners_results_df.set_index('nationality', inplace=True)
winners_results_df.head(10)

,count
nationality,
British,267
German,173
Brazilian,101
French,79
Finnish,49
Italian,43
Austrian,41
Australian,40
Argentine,38


In [38]:
second_results = session.query(drivers_db.nationality, func.count(drivers_db.driverId)).\
    filter(drivers_db.driverId == results_db.driverId).\
    filter(results_db.position == 2).\
    group_by(drivers_db.nationality).\
    order_by(func.count(drivers_db.driverId).desc()).all()
second_results_df = pd.DataFrame(second_results, columns=['nationality', '2nd'])
second_results_df.set_index('nationality', inplace=True)
second_results_df.head(10)

,2nd
nationality,
British,195
German,117
French,111
Brazilian,103
Italian,79
Finnish,67
American,41
Austrian,40
Spanish,39


In [39]:
third_results = session.query(drivers_db.nationality, func.count(drivers_db.driverId)).\
    filter(drivers_db.driverId == results_db.driverId).\
    filter(results_db.position == 3).\
    group_by(drivers_db.nationality).\
    order_by(func.count(drivers_db.driverId).desc()).all()
third_results_df = pd.DataFrame(third_results, columns=['nationality', '3rd'])
third_results_df.set_index('nationality', inplace=True)
third_results_df.head(10)

,3rd
nationality,
British,186
French,113
German,102
Brazilian,89
Italian,85
Finnish,72
American,55
Australian,46
Austrian,37


In [40]:
podium_df = pd.merge(winners_results_df, second_results_df, left_index=True, right_index=True)

In [41]:
podium_df = pd.merge(podium_df, third_results_df, left_index=True, right_index=True)

In [42]:
podium_df['Total Podiums Appearances'] = podium_df.sum(axis=1)
podium_df=podium_df.sort_values('Total Podiums Appearances', ascending=0)
podium_df.head(10)

,count,2nd,3rd,Total Podiums Appearances
nationality,,,,
British,267,195,186,648
German,173,117,102,392
French,79,111,113,303
Brazilian,101,103,89,293
Italian,43,79,85,207
Finnish,49,67,72,188
American,33,41,55,129
Australian,40,39,46,125
Austrian,41,40,37,118
